<a href="https://colab.research.google.com/github/bkvkrll/Introduction-to-Neural-Networks/blob/main/lesson4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Практическое задание**

Вариант 1. 

обучить сверточную нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете fashion-mnist или cifar10

оценить рост точности при увеличении ширины сети (больше ядер)

оценить рост точности при увеличении глубины сети (больше слоев)
(опциционально)сравнить с точностью полносвязной сети для этой выборки

In [16]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!pip install tensorflow -latest


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -l


In [18]:
!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
!pip install mnist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
!pip install tensorflow_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import numpy as np

import pandas as pd

import tensorflow as tf

from tensorflow.keras import models, layers, losses, datasets
from tensorflow.keras.losses import categorical_crossentropy


from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten

from tensorflow.keras.utils import to_categorical
from keras.datasets import fashion_mnist
from tensorflow.keras.datasets import cifar10


from tensorflow.keras.utils import plot_model 

In [22]:
classes = ['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']

In [23]:
batch_size = 512
num_classes = 10
epochs = 10
data_augmentation = False
num_predictions = 20

In [24]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train')
print(X_test.shape[0], 'test')

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

X_train = X_train / 255.0
X_test = X_test / 255.0

X_train shape: (50000, 32, 32, 3)
50000 train
10000 test


In [25]:
model_1 = Sequential([
    Conv2D(16, (5, 5), padding='same', input_shape=X_train.shape[1:], activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:], activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu', input_shape=(13,)),
    Dense(64, activation='linear'),
    Dense(num_classes, activation='softmax'),
])

model_1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 16)        1216      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 16, 16, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 32)        4640      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_3 (Dense)             (None, 256)              

In [26]:
model_1.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          shuffle=True)

Epoch 1/10
98/98 [==============================] - 62s 620ms/step - loss: 1.7688 - accuracy: 0.3695 - val_loss: 1.4772 - val_accuracy: 0.4739
Epoch 2/10
98/98 [==============================] - 59s 604ms/step - loss: 1.4142 - accuracy: 0.4994 - val_loss: 1.3228 - val_accuracy: 0.5363
Epoch 3/10
98/98 [==============================] - 59s 602ms/step - loss: 1.2656 - accuracy: 0.5549 - val_loss: 1.1995 - val_accuracy: 0.5793
Epoch 4/10
98/98 [==============================] - 60s 609ms/step - loss: 1.1664 - accuracy: 0.5891 - val_loss: 1.1411 - val_accuracy: 0.5985
Epoch 5/10
98/98 [==============================] - 60s 612ms/step - loss: 1.0923 - accuracy: 0.6156 - val_loss: 1.1200 - val_accuracy: 0.6062
Epoch 6/10
98/98 [==============================] - 60s 614ms/step - loss: 1.0321 - accuracy: 0.6392 - val_loss: 1.0361 - val_accuracy: 0.6377
Epoch 7/10
98/98 [==============================] - 60s 613ms/step - loss: 0.9808 - accuracy: 0.6565 - val_loss: 1.0387 - val_accuracy: 0.6421

Увеличим количество каналов на конволюционном слое

In [27]:
model_2 = Sequential([
    Conv2D(64, (5, 5), padding='same', input_shape=X_train.shape[1:], activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), padding='same', input_shape=X_train.shape[1:], activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu', input_shape=(13,)),
    Dense(64, activation='linear'),
    Dense(num_classes, activation='softmax'),
])

model_2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 32, 64)        4864      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 8, 8, 128)        0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 8192)              0         
                                                                 
 dense_6 (Dense)             (None, 256)              

In [28]:
model_2.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          shuffle=True)

Epoch 1/10
98/98 [==============================] - 232s 2s/step - loss: 1.7384 - accuracy: 0.3764 - val_loss: 1.4136 - val_accuracy: 0.4914
Epoch 2/10
98/98 [==============================] - 231s 2s/step - loss: 1.2778 - accuracy: 0.5501 - val_loss: 1.2054 - val_accuracy: 0.5723
Epoch 3/10
98/98 [==============================] - 229s 2s/step - loss: 1.0872 - accuracy: 0.6186 - val_loss: 1.0244 - val_accuracy: 0.6419
Epoch 4/10
98/98 [==============================] - 227s 2s/step - loss: 0.9646 - accuracy: 0.6629 - val_loss: 0.9738 - val_accuracy: 0.6588
Epoch 5/10
98/98 [==============================] - 230s 2s/step - loss: 0.8658 - accuracy: 0.6970 - val_loss: 0.9049 - val_accuracy: 0.6860
Epoch 6/10
98/98 [==============================] - 228s 2s/step - loss: 0.7876 - accuracy: 0.7259 - val_loss: 0.8870 - val_accuracy: 0.6960
Epoch 7/10
98/98 [==============================] - 228s 2s/step - loss: 0.7365 - accuracy: 0.7441 - val_loss: 0.9478 - val_accuracy: 0.6723
Epoch 8/10
98

добавим дополнительные слои конволюции и пулинга

In [29]:
model_3 = Sequential([
    Conv2D(16, (5, 5), padding='same', input_shape=X_train.shape[1:], activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:], activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    #ниже дополнительные слои конволюции и пулинга
    Conv2D(64, (3, 3), padding='same', input_shape=X_train.shape[1:], activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), padding='same', input_shape=X_train.shape[1:], activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(256, activation='relu', input_shape=(13,)),
    Dense(64, activation='linear'),
    Dense(num_classes, activation='softmax'),
])

model_3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 32, 32, 16)        1216      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 16, 16, 16)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 32)        4640      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 8, 8, 64)          18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 4, 4, 64)        

In [30]:
model_3.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          shuffle=True)

Epoch 1/10
98/98 [==============================] - 78s 785ms/step - loss: 1.8340 - accuracy: 0.3312 - val_loss: 1.5273 - val_accuracy: 0.4419
Epoch 2/10
98/98 [==============================] - 77s 784ms/step - loss: 1.4655 - accuracy: 0.4674 - val_loss: 1.3696 - val_accuracy: 0.5105
Epoch 3/10
98/98 [==============================] - 77s 785ms/step - loss: 1.2944 - accuracy: 0.5345 - val_loss: 1.2042 - val_accuracy: 0.5702
Epoch 4/10
98/98 [==============================] - 76s 781ms/step - loss: 1.1682 - accuracy: 0.5839 - val_loss: 1.1632 - val_accuracy: 0.5842
Epoch 5/10
98/98 [==============================] - 77s 784ms/step - loss: 1.0694 - accuracy: 0.6212 - val_loss: 1.0520 - val_accuracy: 0.6280
Epoch 6/10
98/98 [==============================] - 77s 786ms/step - loss: 1.0195 - accuracy: 0.6379 - val_loss: 1.0656 - val_accuracy: 0.6250
Epoch 7/10
98/98 [==============================] - 78s 798ms/step - loss: 0.9438 - accuracy: 0.6660 - val_loss: 0.9805 - val_accuracy: 0.6593

Выводы: при увеличении количества каналов на конволюционном слое метрика выросла с 0.6648 до 0.7149, но это сопровождалось существенным увеличение времени обучения. При добавлении дополнительных слоев метрика изменилась не существенно